In [40]:
using JuMP
struct Bus
    idx:: Int64
    name:: String
    p:: Matrix{Float64}
    max:: Float64
end
x = [2 3;
     4 5]
NODE = [Bus(1, "lima", x, 4), Bus(1, "lima", x, 5)]  
N = 1:2

1:2

In [41]:
m = Model()
@variable(m, P[N])
@constraint(m, c[n in N], P[n] <= NODE[n].max)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c[1] : P[1] ≤ 4
 c[2] : P[2] ≤ 5

In [6]:
using CSV, DataFrames, NamedArrays

function to_Structs(structure::DataType, inputs_dir:: String, filename:: String):: Vector{structure}
    file_dir = joinpath(inputs_dir, filename)
    struct_names = fieldnames(structure)
    struct_types = fieldtypes(structure)

    first_csvlines = CSV.File(joinpath(inputs_dir, filename); limit=1)
    csv_header = Tuple(propertynames(first_csvlines))

    @assert csv_header == struct_names """Column names of $filename does not match the fields of the structure $structure."""

    df = CSV.read(file_dir, DataFrame; types=Dict(zip(struct_names, struct_types)))
    cols = Tuple(df[!, col] for col in names(df))
    V = structure.(cols...)    
    
    return V
end

function to_multidim_NamedArray(structures:: Vector{T}, dims:: Vector{Symbol}, value:: Symbol):: NamedArray{Union{Missing, Float64}} where {T} 
  
    vals_in_dim = [unique(getfield.(structures, d)) for (i, d) in enumerate(dims)]
    
    arr = Array{Union{Missing, Float64}}(missing, length.(vals_in_dim)...)
    arr = NamedArray(arr, vals_in_dim, dims)

    for s in structures
        arr[getfield.(Ref(s), dims)...] = getfield(s, value)
    end

    return arr
end

to_multidim_NamedArray (generic function with 1 method)

In [3]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "buses.csv"
file_view = CSV.File(joinpath(inputs_dir, filename); limit=1)
csv_header = Tuple(propertynames(file_view))

(:bus_id, :kv, :type, :lat, :lon, :slack)

In [5]:
fieldnames(Bus)

(:bus_id, :kv, :type, :lat, :lon, :slack)

In [49]:
struct Bus
    bus_id:: String
    kv:: Float64
    type:: String
    lat:: Float64
    lon:: Float64
    slack:: Bool
end


In [52]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "buses.csv"

    # Get a list of Bus structures
    buses = to_Structs(Bus, inputs_dir, "buses.csv")

    # Get a list of the bus IDs
    N = getfield.(buses, :bus_id)

    # Transform buses into NamedArray, so we can access buses by their IDs
    buses = NamedArray(buses, (N))

    # Get slack bus
    slack_bus = buses[ findfirst([n.slack == true for n in buses]) ]

Bus("lima", 120.0, "Substation", -50.0, 550.0, true)

In [12]:
struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

LOADS = to_Structs(Load, inputs_dir, filename)
arr = to_multidim_NamedArray(LOADS, [:bus_id, :sc_id, :t_id], :load)

2×2×3 Named Array{Union{Missing, Float64}, 3}

[:, :, t_id=1] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     4.0  missing
lima           │ missing      2.0

[:, :, t_id=2] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     5.0  missing
lima           │ missing      1.0

[:, :, t_id=3] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     6.0  missing
lima           │ missing      4.0

In [13]:
struct CapacityFactor
    gen_id:: String
    tp_id:: String
    sc_id:: String
    capacity_factor:: Float64
end

In [36]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "capacity_factors.csv"
CF = to_Structs(CapacityFactor, inputs_dir, filename)
cf = to_multidim_NamedArray(CF, [:gen_id, :tp_id, :sc_id], :capacity_factor)

2×3×2 Named Array{Union{Missing, Float64}, 3}

[:, :, sc_id=low] =
gen_id ╲ tp_id │   1    2    3
───────────────┼──────────────
sun_lim        │ 0.1  0.2  3.0
winds          │ 0.5  1.0  0.2

[:, :, sc_id=high] =
gen_id ╲ tp_id │   1    2    3
───────────────┼──────────────
sun_lim        │ 0.1  0.2  3.0
winds          │ 0.5  0.2  1.0

In [8]:
S = LOADS

dims = [:bus_id, :sc_id, :t_id]
val = :load

vals_in_dim = [unique(getfield.(S, d)) for (i, d) in enumerate(dims)]

arr = Array{Union{Missing, Float64}}(missing, length.(vals_in_dim)...)
arr = NamedArray(arr, vals_in_dim, dims)
for s in S
    arr[getfield.(Ref(s), dims)...] = getfield(s, val)
end
arr

2×2×3 Named Array{Union{Missing, Float64}, 3}

[:, :, t_id=1] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     4.0  missing
lima           │ missing      2.0

[:, :, t_id=2] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     5.0  missing
lima           │ missing      1.0

[:, :, t_id=3] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     6.0  missing
lima           │ missing      4.0

In [10]:
arr["sd", "low", 1]

4.0

In [46]:
struct Generator
    gen_id:: String
    gen_tech:: String
    bus_id:: String
    c2:: Float64
    c1:: Float64
    c0:: Float64
    invest_cost:: Float64
    exist_cap:: Float64
    cap_limit:: Float64
    var_om_cost:: Float64
end

struct Storage 
    sto_id:: String
    sto_tech:: String
    bus_id:: String
    invest_cost:: Float64
    exist_power_cap:: Float64
    exist_energy_cap:: Float64
    var_om_cost:: Float64
    efficiency:: Float64
    duration:: Float64
end

In [48]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "generators.csv"
gens = to_Structs(Generator, inputs_dir, filename)
G = getfield.(gens, :gen_id)
gens = NamedArray(gens, (G))

3-element Named Vector{Generator}
A       │ 
────────┼──────────────────────────────────────────────────────────────────────────
sun_lim │ Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5)
winds   │    Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)
dirt    │       Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)

In [57]:
GENS_AT_BUS = NamedArray( [ gens[ findall([g.bus_id == n for g in gens]) ] for n in N], (N) )


2-element Named Vector{NamedVector{Generator, Vector{Generator}, Tuple{OrderedCollections.OrderedDict{String, Int64}}}}
A    │ 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
sd   │                                                                               Generator[Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)]
lima │ Generator[Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5), Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)]

In [59]:
GENS_AT_BUS["lima"]

2-element Named Vector{Generator}
A       │ 
────────┼──────────────────────────────────────────────────────────────────────────
sun_lim │ Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5)
winds   │    Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)

In [44]:
GV_ids = names(cf, 1)
GV = G[GV_ids]

GN_ids = setdiff(G_ids, GV_ids)
GN = G[GN_ids]

GV_AT_BUS = Dict(n => intersect(GV,gens) for (n, gens) in GENS_AT_BUS)



1-element Named Vector{Generator}
A    │ 
─────┼────────────────────────────────────────────────────────────────────
dirt │ Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)

In [20]:
using CSV, DataFrames

file = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs/scenarios.csv"
df = CSV.read(file, DataFrame)
cols = Tuple(df[!, col] for col in names(df))
S = Scenario.(cols...)


2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [19]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "scenarios.csv"
SX = to_vec_of_structs(Scenario, inputs_dir, filename)

2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [50]:
struct Load
    bus_id:: Int64
    sc_id:: Int64
    t_id:: Int64
    value:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
L = to_vec_of_structs(Load, inputs_dir, filename)

In [84]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
load = CSV.read(joinpath(inputs_dir, "loads.csv"),DataFrame, 
types=[Int64, Int64, Int64, Float64]);
load

to_Structs (generic function with 2 methods)

In [88]:
using AxisKeys
x = wrapdims(load, :value, :bus_id, :scenario, :t_id)

(:bus_id, :sc_id, :t_id, :load)

In [ ]:
using CSV, DataFrames, NamedArrays

inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end


#V = to_Structs(Load, inputs_dir, filename)

MethodError: MethodError: objects of type NTuple{4, Symbol} are not callable
The object of type `NTuple{4, Symbol}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [65]:
using AxisKeys

# Your 2D matrix
data_matrix = [1 1 10;
               2 2 20;
               3 1 30;
               1 2 40;
               2 1 200;
               3 2 100]

# Extract indices and values
indices = data_matrix[:, 1:2]
values = data_matrix[:, 3]

# Determine unique keys for each dimension
key1_values = unique(indices[:, 1])
key2_values = unique(indices[:, 2])

# Reshape values and create AxisArray
x = wrapdims(reshape(values, length(key1_values), length(key2_values)), key1_values, key2_values)



3-element Vector{String}:
 "sd"
 "low"
 "1"

In [67]:
reshape(values, length(key1_values), length(key2_values))